# Natural language processing: The bag-of-words algorithm 

files needed = ('reviews.csv', 'newsgroups.zip')

In this lecture we're going to shift gears from dealing with numerical data to text data. 

Working with text as data is known as Natural Language Processing (NLP). A common use of NLP is categorizing a set of text. Perhaps the most ubiquitous example is a spam filter. It reads the text of a message and determines if it is "spam" or "ham." Another common use case is to deduce the sentiment of a block of text. Is the earnings call positive or negative? Is the Fed statement hawkish or dovish? Is the review positive or negative? NLP models form the basis for automatic translation software and voice recognition. Lots of cool stuff.  

We'll employ one simple NLP algorithm known as the *bag-of-words* algorithm to deduce the sentiment of a dataset of movie reviews. There are more sophisticated methods but this will give us the big idea. "More sophisticated" typically amounts to tweaks on how we process the data or more complex classifier (discrete) models.

Our work today falls under the category of *supervised* machine learning, in that we are training the algorithm/estimating the model on *labeled* data where the true classification is known. 

## NLP vs. classifiers

Natural language is harder for algorithms to interpret and analyze than numeric data since:

1. Sentences are not of fixed lengths, but most algorithms require a standard input vector size.

2. Most algorithms cannot understand words as input, so each word needs to be represented by some numeric value.

So our method is:

1. Pre-processing: Clean up the text. 
2. Estimate a model on the training data: Let $\text{word}_{ji}$ be the number of times the word $j$ occurs in review $i$
$$\text{Pr}(\text{review}_{i}=1|\text{words}) = \text{logit}(\beta_{0} + \beta_{1}\text{word}_{1i}+ \beta_{2}\text{word}_{2i}+\cdots)$$ 
3. Use the estimated model to classify new reviews.

*I'm using the machine-learning term "classifier" in this notebook, but everywhere you see the word "classifier," you can replace it with "Logit" and nothing changes. Similarly, you can replace the word "features" with "exogenous variables."*

In [1]:
import numpy as np
import pandas as pd

## A simple example with SPAM
Spam emails/messages belong to the broad category of unsolicited messages received by a user. Spam occupies unwanted space and bandwidth, amplifies the threat of viruses, and in general exploits a user's connection to social networks. Plus, they're annoying.

Three messages. 

In [2]:
# Corpus is a fancy word for a collection (or a body) of text.
# Label marks a message as spam (1) or not spam (0).

corpus = [('Text 1', 'You have won a prize. Call today to claim.', 1),
          ('Text 2', 'It is your mother. Call me.', 0),
          ('Text 3', 'Are you around today? I need a favor from you.', 1)]
data = pd.DataFrame(corpus, columns=['Document Number','Text of Documents', 'Label'])
data.head()

,Document Number,Text of Documents,Label
0,Text 1,You have won a prize. Call today to claim.,1
1,Text 2,It is your mother. Call me.,0
2,Text 3,Are you around today? I need a favor from you.,1


Even though python is good with text, we will still need to convert our text into  numeric data to get a classifier model to analyze it. Let's create a matrix with the word counts. Each row of the matrix is an observation (a message) and each column is a word. The cells in the matrix are the number of times that word is found in the message.

The scikit package gives us the `CountVectorizer` to do this for us. 

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['Text of Documents'])

print(X.toarray())

[[0 0 1 1 0 0 1 0 0 0 0 0 1 1 1 1 1 0]
 [0 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1]
 [1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 2 0]]


Note that `X` is an array-like object. We are in the realm of scikit, which doesn't use DataFrames. Let's turn this back into a DataFrame, though, so we can see things clearly.

In [4]:
cols = vectorizer.get_feature_names_out()

exog = pd.DataFrame(X.toarray(), columns=cols, index=data['Document Number'])

exog.head()

,are,around,call,claim,favor,from,have,is,it,me,mother,need,prize,to,today,won,you,your
Document Number,,,,,,,,,,,,,,,,,,
Text 1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,1,1,1,0
Text 2,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1
Text 3,1,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,2,0


The `exog` DataFrame contains our features and the `data['Label']` column contains our outcome variable. We now have the data ready  to estimate a classifier model (e.g., a logit regression).

$$\text{Pr(Label=1|exog)} = \Lambda( \beta_0 + \beta_1\text{are} + \beta_1\text{are} + \beta_1\text{around} + \beta_1\text{call} + \cdots)$$

In statsmodels, this would be:

```
sm.Logit(data['label'], exog).fit()
```

This dataset is too small to actually fit a model, so let's move on to something bigger. 


Note that this methodology of turning text into data is not limited to classification problems. For example, we could use this approach to connect stock performance with Federal Open Market Committee (FOMC) statements to predict how the Federal Reserve's statements on the economy influence the S&P500, the Dow Jones, individual stocks, and government treasury prices. NLP is a broad topic and a lot of fun.

## The new package

Text data have a whole set of problems to deal with: misspelling, different versions of words, capitalization.
We will use the *natural language tool kit* (nltk) to help us process the text data. It comes with anaconda, but if you need to install it: 

```python
pip install --user nltk
```

In [5]:
import nltk

## Detecting sentiment

The dataset that we are using is a large movie review dataset from [here](http://ai.stanford.edu/~amaas/data/sentiment/). It contains positive and negative reviews, in English. There are two columns. The first column tells us whether the review is positive (1) or negative (0). The second column corresponds to the text of the review. 

The full dataset at the above source contains 50,000 reviews, half of them positive, if you'd like to play around with these methods more. The authors below also created separate train and test sets with a disjoint set of movies so that performance wouldn't be juiced upward by movie-specific terms like "Chewbacca" or "Romy". We'll skip the train/test distinction in the below, but note that model performance should be tested out-of-sample. We'll work with a 10% random sample.

Source reference: Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).

In [6]:
dataset = pd.read_csv('reviews.csv', usecols=['label','review'])
dataset.head(5)

,label,review
0,1,As much as I hate to disagree with the origina...
1,1,"Between sweeping, extraordinary scenes within ..."
2,1,"This is the ultimate, and I mean the ULTIMATE,..."
3,0,Recognizing the picture of the diner on the co...
4,0,I have nothing at all against Paul Schrader. I...


## Data Pre-Processing

This is the part that makes nlp different from working with numeric data. We need to clean up the text and turn it into a feature matrix. 

```
'I am helping raise $100 for UW Madison'                     original
'i am helping raise $100 for uw madison'                     homogenize capitalization
'i am helping raise for uw madison'                          remove non-alphabetic characters
['i', 'am', 'helping', 'raise',  'for', 'uw', 'madison']     tokenize
['helping', 'raise', 'uw', 'madison']                        remove stop words
['help', 'raise', 'uw', 'madison']                           stem and lem
'help raise uw madison'                                      back to a single string
```

Then create the feature matrix

| help | raise | uw | madison|
|------|-------|----|--------|
| 1    | 1     |  1 |     1  |



### 1. Homogenize the capitalization

We don't want to worry about 'Hello' not being equal to 'hello'. Let's make everything lowercase. 

In [7]:
# 1. Homogenize capitalization
dataset['review'] = dataset['review'].str.lower()
dataset.head()

,label,review
0,1,as much as i hate to disagree with the origina...
1,1,"between sweeping, extraordinary scenes within ..."
2,1,"this is the ultimate, and i mean the ultimate,..."
3,0,recognizing the picture of the diner on the co...
4,0,i have nothing at all against paul schrader. i...


### 2. Remove non-alphabetic characters

Our algorithm will only use words to characterize reviews. This is not strictly necessary, but can help sometimes. 

We will remove them using a regular expression. 

As a reminder, the code to remove the non-alphabetic characters is 
```python
dataset['review'].str.replace('[^A-Za-z]', ' ', regex=True)
```

The regex part is the `'[^A-Za-z]'`. It says: "find everything that is not the letters A through Z or a through z." We replace the non-alphabetic stuff with a space.

In [8]:
# 2. Remove non-alphabetic characters
dataset['review'] = dataset['review'].str.replace('[^A-Za-z]', ' ', regex=True)
dataset.head()

,label,review
0,1,as much as i hate to disagree with the origina...
1,1,between sweeping extraordinary scenes within ...
2,1,this is the ultimate and i mean the ultimate ...
3,0,recognizing the picture of the diner on the co...
4,0,i have nothing at all against paul schrader i...


### 3. Tokenize the strings

Break the strings up into lists of words, which are easier to process. This is very similar to using `.str.split(' ')`. Here we use the tokenizer method from nltk. It is a bit more sophisticated than a simple split.

```python
from nltk.tokenize import word_tokenize as wt 
```

We also need to download the punctuation dataset. 

In [9]:
# 3. Tokenize the strings.

# Get the punctuation. 
nltk.download('punkt')

from nltk.tokenize import word_tokenize as wt 
dataset['review'] = dataset['review'].apply(wt)
dataset.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ashleyswanson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,label,review
0,1,"[as, much, as, i, hate, to, disagree, with, th..."
1,1,"[between, sweeping, extraordinary, scenes, wit..."
2,1,"[this, is, the, ultimate, and, i, mean, the, u..."
3,0,"[recognizing, the, picture, of, the, diner, on..."
4,0,"[i, have, nothing, at, all, against, paul, sch..."


### 4. Removing stop words

Now we eliminate *stop words*&mdash;words in the text that add no specific meaning. They often involve prepositions, helping verbs, and articles (e.g., in, the, an, is). Since these add no value to our model, let's get rid of them.

Fortunately, linguists have already identified stopwords specific to each language, so we can readily identify and exclude them.

```python
from nltk.corpus import stopwords
stop_wrds = stopwords.words('english')
```

`stop_wrds` is a list of English-language stop words. 

We need to loop through the lists and check for stop words. I will write a small function that does the looping and then apply it to the DataFrame's column using `.apply()`.

Again, we need to download the stopwords first. 

In [10]:
# 4. Remove stop words.
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_wrds = stopwords.words('english')

def remove_stops(x):
    temp = []
    for word in x:
        if word not in stop_wrds:
            temp.append(word)
    return temp

dataset['review'] = dataset['review'].apply(remove_stops)
dataset.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ashleyswanson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,review
0,1,"[much, hate, disagree, original, poster, found..."
1,1,"[sweeping, extraordinary, scenes, within, plet..."
2,1,"[ultimate, mean, ultimate, adventure, classic,..."
3,0,"[recognizing, picture, diner, cover, dvd, made..."
4,0,"[nothing, paul, schrader, fact, hardcore, one,..."


### 5. Stemming and lemmatization
Words like act, actor, and acting are all versions of the same root word (act). **Stemming** and **lemmatization** are techniques used to truncate words in order to get the stem or the base word. The difference between these two methods is that after stemming, the stem may not be an actual word, whereas lemmatization always produces a real word, which results in better interpretation of the corpora by humans.

For example, studies could be stemmed as studi (not a word), but will be lemmatized as study (an existing word). To be honest, this feels like a rabbit hole, so I'm treating this stuff as a black box and trusting that the linguists are doing a good job.

Let's stem these words.

In [11]:
# 5. Stemming and lemmatization
from nltk.stem.porter import PorterStemmer

def stem_it(x):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in x]

dataset['review'] = dataset['review'].apply(stem_it) 
dataset.head()

,label,review
0,1,"[much, hate, disagre, origin, poster, found, a..."
1,1,"[sweep, extraordinari, scene, within, plethora..."
2,1,"[ultim, mean, ultim, adventur, classic, plot, ..."
3,0,"[recogn, pictur, diner, cover, dvd, made, real..."
4,0,"[noth, paul, schrader, fact, hardcor, one, fav..."


That seemed like a lot of work, but it always does when we are first learning something. Putting all the code together, the processing is simply: 

```python
dataset['review'] = dataset['review'].str.lower()
dataset['review'] = dataset['review'].str.replace('[^A-Za-z]', ' ', regex=True)
dataset['review'] = dataset['review'].apply(wt)
dataset['review'] = dataset['review'].apply(remove_stops)
dataset['review'] = dataset['review'].apply(stem_it) 
```

You could even wrap all that up in a function, too...


### Create the feature matrix

We are done pre-processing. 

1. Turn the lists of words back into strings.
2. Create the feature matrix using `CountVectorizer`.

In [12]:
dataset['review'] = dataset['review'].str.join(' ')

In [13]:
# The matrix of word counts. I am limiting the feature matrix to 250 columns.

X = CountVectorizer(max_features=250).fit_transform(dataset['review'])
exog = pd.DataFrame(X.toarray())

# The outcome data.
endog = dataset['label']

### Estimate the logit model

I'm using the scikit learn package to estimate the logit model. This is estimating the same logit model as we did with statsmodels, but scikit learn has a faster implementation. 

The syntax is a bit different, but it's the same idea as what we saw in class Tuesday.

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
res_rev = LogisticRegression(random_state=0, max_iter=1000).fit(exog, endog)

scikit does not have a `.summary()` function like statsmodels.  In this kind of application, we are not concerned with the individual coefficients. We don't really care *why* a word is good at classifying a positive review, we just care that the model works.  That is, we care about prediction, not identification of an underlying relationship. 

You can still recover all the parameters, etc., but not in a neat table. 

How is our in-sample fit? Compare the predicted values to the actual values. The `.score()` method computes the share of the reviews properly categorized. 

In [16]:
print('{0:.1f}% of sample reviews were properly categorized.'.format(res_rev.score(exog, endog)*100))

84.4% of sample reviews were properly categorized.


The confusion matrix is (from the [docs](https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.MultinomialResults.pred_table.html)):

```
pred_table[i,j] refers to the number of times “i” was observed and the model predicted “j”. 
Correct predictions are along the diagonal.
```

In [17]:
from sklearn.metrics import confusion_matrix
#confusion_matrix(endog, res_rev.predict(exog))
pd.DataFrame(confusion_matrix(endog, res_rev.predict(exog)))

,0,1
0,1014,215
1,175,1096


We have 175 false negatives (14%), and 215 false positives (17%). Better than random, but performance may improve with a larger dataset and more features. May also be helpful to strip out proper names (of actors/directors/writers/titles) in pre-processing.

## Top Hat Practice Exercise: Newsgroups

We're going to practice using the "20 Newsgroup" data set, which is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of my knowledge, it was originally collected by Ken Lang, probably for his "Newsweeder: Learning to filter netnews" paper, though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.

The goal is to categorize each post based on its content. 

1. Download, unzip, and load the file 'newsgroups.csv'. Only import the first 500 rows. Try the `nrows` option of `.read_csv()`. 

   `article` is the message. `category code` is the newsgroup category code. `category` is the newsgroup category name. 
   
   **Our goal:** Create a classifier that predicts the category code of an article. 

In [18]:
news = pd.read_csv('newsgroups.csv', nrows=500)
news.head()

,article,category code,category
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space


2. How many articles are there in each category. Looks like it's time for `.groupby()`.

In [19]:
#news['category'].value_counts()
news.groupby('category')['article'].count()

category
alt.atheism                 26
comp.graphics               23
comp.os.ms-windows.misc     36
comp.sys.ibm.pc.hardware    18
comp.sys.mac.hardware       34
comp.windows.x              22
misc.forsale                27
rec.autos                   21
rec.motorcycles             29
rec.sport.baseball          28
rec.sport.hockey            25
sci.crypt                   20
sci.electronics             30
sci.med                     30
sci.space                   27
soc.religion.christian      32
talk.politics.guns          19
talk.politics.mideast       26
talk.politics.misc          18
talk.religion.misc           9
Name: article, dtype: int64

3. Process the text data. All the code to do this is gathered in the cell above the **Create feature matrix** heading above. 

In [20]:
news['article'] = news['article'].str.lower()
news['article'] = news['article'].str.replace('[^A-Za-z]', ' ', regex=True)
news['article'] = news['article'].apply(wt)
news['article'] = news['article'].apply(remove_stops)
news['article'] = news['article'].apply(stem_it) 
news.head()

,article,category code,category
0,"[wonder, anyon, could, enlighten, car, saw, da...",7,rec.autos
1,"[fair, number, brave, soul, upgrad, si, clock,...",4,comp.sys.mac.hardware
2,"[well, folk, mac, plu, final, gave, ghost, wee...",4,comp.sys.mac.hardware
3,"[weitek, address, phone, number, like, get, in...",1,comp.graphics
4,"[articl, c, owcb, n, p, world, std, com, tomba...",14,sci.space


4. Turn the lists of words in `article` into strings. 
5. Create the feature matrix. Allow it to use 100 features.
6. Create the outcome variable. (the Series that contains the category codes)

In [21]:
news['article'] = news['article'].str.join(' ')
X = CountVectorizer(max_features=100).fit_transform(news['article'])
exog = pd.DataFrame(X.toarray())

# The outcome data.
endog = news['category code']

7. Estimate the logit model.
8. Use `.score()` to check the in-sample fit.
9. Compute the confusion matrix.

In [22]:
res_news = LogisticRegression(random_state=0, max_iter=1000).fit(exog, endog)
res_news.score(exog, endog)*100

76.2

In [23]:
pd.DataFrame(confusion_matrix(endog, res_news.predict(exog)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,20,1,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
1,2,14,1,0,0,0,1,0,0,1,2,0,1,0,1,0,0,0,0,0
2,0,0,29,0,0,0,1,1,1,2,0,0,1,1,0,0,0,0,0,0
3,0,0,0,15,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0
4,0,1,3,0,21,1,0,0,0,4,0,0,2,1,0,0,0,0,1,0
5,0,1,0,0,2,16,0,0,0,2,0,0,1,0,0,0,0,0,0,0
6,0,0,0,0,1,0,25,0,0,1,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,1,14,0,2,0,0,3,0,0,0,0,0,0,0
8,0,0,0,0,1,1,0,0,22,3,1,0,0,0,0,0,0,1,0,0
9,0,0,0,0,1,0,0,0,0,23,1,0,0,1,1,0,0,1,0,0


10. Go back to step 1. and increase the number of rows to 1000. Rerun your code. Does the accuracy improve?

    Go back to step 5. and add more features to your exogenous variables. Does the accuracy improve?
    
    Do you see any patterns in the confusion matrix?

In [24]:
news = pd.read_csv('newsgroups.csv', nrows=1000)

news['article'] = news['article'].str.lower()
news['article'] = news['article'].str.replace('[^A-Za-z]', ' ', regex=True)
news['article'] = news['article'].apply(wt)
news['article'] = news['article'].apply(remove_stops)
news['article'] = news['article'].apply(stem_it) 

news['article'] = news['article'].str.join(' ')
X = CountVectorizer(max_features=100).fit_transform(news['article'])
exog = pd.DataFrame(X.toarray())

# The outcome data.
endog = news['category code']
res_news = LogisticRegression(random_state=0, max_iter=1000).fit(exog, endog)
res_news.score(exog, endog)*100

63.5

In [25]:
news = pd.read_csv('newsgroups.csv', nrows=1000)

news['article'] = news['article'].str.lower()
news['article'] = news['article'].str.replace('[^A-Za-z]', ' ', regex=True)
news['article'] = news['article'].apply(wt)
news['article'] = news['article'].apply(remove_stops)
news['article'] = news['article'].apply(stem_it) 

news['article'] = news['article'].str.join(' ')
X = CountVectorizer(max_features=200).fit_transform(news['article'])
exog = pd.DataFrame(X.toarray())

# The outcome data.
endog = news['category code']
res_news = LogisticRegression(random_state=0, max_iter=1000).fit(exog, endog)
res_news.score(exog, endog)*100

86.1

In [26]:
pd.DataFrame(confusion_matrix(endog, res_news.predict(exog)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,47,1,0,0,1,0,0,0,0,3,0,0,0,1,0,0,0,1,0,0
1,0,37,0,1,2,0,2,0,1,0,0,0,1,1,0,0,0,0,0,0
2,0,0,45,2,1,0,1,0,1,2,1,0,1,0,0,0,0,0,1,0
3,1,0,1,36,3,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0
4,0,0,0,1,51,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0
5,0,1,2,0,4,36,1,0,0,2,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,52,0,0,2,1,0,1,0,0,0,0,0,0,0
7,0,0,0,0,1,0,1,45,0,2,0,0,3,0,0,0,0,0,1,0
8,0,0,0,0,1,1,1,0,37,2,2,0,5,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,51,1,0,0,0,0,0,0,1,0,0


In [27]:
news[news['category code']==9].head(1)

,article,category code,category
26,doug robert ken hill nl mvp let go spo,9,rec.sport.baseball


## Want more?

This notebook was meant to give you an idea of how to turn words into data. We used a very simple "bag-of-words" model where we only considered how often a word appears. There are more complex methods (term frequency-inverse document frequency, ngrams,...) and alternative modeling (machine learning methods). 

If you want to learn more, Dan Jurafsky and James H. Martin have a [free book online](https://web.stanford.edu/~jurafsky/slp3/) that I think is quite good. As always, you can google "nlp tutorial" and see what is out there. 